In [1]:
import os, shutil
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import PIL
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import matplotlib

In [2]:
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [4]:
import sys
sys.path.append('../../')
import OtolReader as otolr

## Image classification results

In [6]:
basedir = '../../OtolReader-Publication/Models'
with open(os.path.join(basedir, 'crossval_order.p'), 'rb') as f:
    cvorder = np.array(pickle.load(f))
pdir = os.path.join(basedir, 'CrossVal-Binary-5classes-30Images')
model_prefix = 'binarynet_'
nim_fold = 15
sampledir = os.path.join(basedir, 'CrossVal-ClassifierSampleTable')
samps_prefix = 'ClassifierNetworkForSelection_SmoothedSamples_'
retrained_dir = os.path.join(basedir, 'Retrained-Binary-Nets')
classmod_name_base = os.path.join(basedir, 'CrossVal-Classifier-5classes-30Images', 'classnet_')
imdir = '../../Images/OtolithImages/'
mark_list = ['3,5H10', '1,6H', 'none', '6,2H', '4n,2n,2H']
fmod_name_base = os.path.join(retrained_dir, 'binarynet_')

In [28]:
save_scores = []
y = []
for fold_ind in range(10):
    with open(os.path.join(sampledir, samps_prefix + str(fold_ind) + '.p'), 'rb') as f:
        train_array = pickle.load(f)
    nm_model = keras.models.load_model(fmod_name_base + str(fold_ind) + '.h5', compile=False)
    for ind in range(fold_ind * nim_fold, (fold_ind + 1) * nim_fold):
        samps = train_array[ind]
        mark_ind, im_ind = otolr.OtolithAnalysis.feature_functions.fcn_mark_im_ind(cvorder[ind], 30)
        if mark_ind == 2:
            y.append(0)
        else:
            y.append(1)
        scores = np.mean(nm_model.predict(np.expand_dims(samps, axis=2)), axis=0)
        save_scores.append(scores)

In [42]:
yhat = []
for s in save_scores:
    if s[0] > 0.2:
        yhat.append(0)
    else:
        yhat.append(1)
acc = np.sum(np.array(yhat)==np.array(y))/len(yhat)
print('Binary accuracy at expected optimal cutoff: {:0.3f}'.format(acc))

Binary accuracy at expected optimal cutoff: 0.987


In [43]:
scbase = os.path.join(basedir, 'CrossVal-ClassifierSampleTable', 'ClassifierNetworkForSelection_Scores_')
yhatclass = np.ones(150, dtype=int) * -1
yhatfin = np.ones(150, dtype=int) * 2
yfin = np.ones(150, dtype=int) * -1
for fold_ind in range(10):
    with open(scbase + str(fold_ind) + '.p', 'rb') as f:
        sc = pickle.load(f)
    for ind in range(nim_fold * fold_ind, (fold_ind + 1) * nim_fold):
        mark, im_ind = otolr.OtolithAnalysis.feature_functions.fcn_mark_im_ind(cvorder[ind], 30)
        sctemp = np.average(sc[ind], axis=0)
        yhatclass[ind] = np.argmax(sctemp)
        if yhat[ind] != 0:
            yhatfin[ind] = yhatclass[ind] 
        yfin[ind] = mark
print('Overall classification accuracy: {:0.3f}'.format(np.sum(yhatfin==yfin) / len(yfin)))

Overall classification accuracy: 0.953


In [44]:
conf = np.zeros([5, 5])
for ind in range(len(yfin)):
    conf[yfin[ind], yhatfin[ind]] += 1
print(conf)

[[28.  1.  0.  0.  1.]
 [ 0. 29.  1.  0.  0.]
 [ 0.  0. 30.  0.  0.]
 [ 0.  0.  1. 29.  0.]
 [ 1.  2.  0.  0. 27.]]


In [9]:
# test the image classfication function
nm_model = keras.models.load_model(fmod_name_base + str(0) + '.h5', compile=False)
class_model = keras.models.load_model(classmod_name_base + str(0) + '.h5', compile=False)
im_ind = cvorder[0]
print("Evaluating image #{:0.0f}".format(im_ind))
mark_ind, classed_im_ind = otolr.OtolithAnalysis.feature_functions.fcn_mark_im_ind(im_ind, 30)
print("Expected mark: {}".format(mark_list[mark_ind]))
im_path = os.path.join(imdir, mark_list[mark_ind], str(classed_im_ind) + '.jpg')
mark_hat = otolr.OtolithAnalysis.im_classifier.classify_image(im_path, class_model, nm_model, 0.2)
print("Estimated mark: {}".format(mark_hat))

Evaluating image #14
Expected mark: 3,5H10
Estimated mark: 3,5H10


### Full test set

In [128]:
basedir = '../../Network classifier/TwoNetworkTesting/FullTrainingSetModels'
dir0 = 'ClassifierSampleSelection'
subdir1 = 'Samples'
subdir2 = 'Scores'
subdir3 = 'Score_Tables'
dir1 = 'ClassifierSampleSelection50'
pstr = '2019_08_18_ClassifierNetworkForSelection_'
destdir = 'ClassifierSampleSelectionCombined'

In [129]:
train_path = os.path.join(basedir, '2019_08_26_ClassifierNetworkForSelection_SmoothedSamples_0.p')
test_path = os.path.join(basedir, 'TestSetSamples/2019_08_26_ClassifierNetworkForSelection_SmoothedSamples_0.p')
def make_train_array(samp_path):
    with open(samp_path, 'rb') as f:
        train_array = pickle.load(f)
    tr = []
    tr_labs = []
    for t_ind in range(len(train_array)):
        tr.extend(train_array[t_ind])
        mark_ind, im_ind = fcn_mark_im_ind(t_ind, 30)
        if mark_ind == 2:
            lab = 0
        else:
            lab = 1
        tr_labs.extend([lab for _ in range(len(train_array[t_ind]))])
    return tr, tr_labs, train_array

def fcn_mark_im_ind(im_ind_raw, n_img):
    """
    Calculates the mark an image index based on the total image index
    For example, im_ind_raw is 52 and n_img is 30, then mark_ind is 1
    and im_ind is 22
    """
    mark_ind = int(im_ind_raw/n_img)
    im_ind = im_ind_raw - mark_ind * n_img
    return mark_ind, im_ind

In [131]:
y = []
yhat = []
save_scores = []
fmod_name_base = os.path.join(basedir, 'binarynet_')
print("Currently evaluating fold number {}".format('fulltraining'))
mod = keras.models.load_model(fmod_name_base + 'fulltraining.h5', compile=False)
temp_model = make_transfer_module(mod)
tr, tr_labs, train_array = make_train_array(train_path)
while np.mean(tr_labs) > 0.5:
    none_im_ind = np.random.randint(0, 30)
    tr_ind = none_im_ind + 60
    tr.extend(train_array[tr_ind])
    tr_labs.extend([ 0 for _ in range(len(train_array[tr_ind]))])
tr = np.array(tr)
tr_labs = np.array(tr_labs, dtype=int)
tr_2 = temp_model.predict(np.expand_dims(tr, axis=2))
nm_model = make_none_marked_model()
nm_model.fit(np.expand_dims(tr_2, axis=2), tr_labs, epochs=20)
print('check 1')
_, _, test_array = make_train_array(test_path)
for ind in range(len(test_array)):
    samps = test_array[ind]
    samps_2 = temp_model.predict(np.expand_dims(samps, axis=2))
    mark_ind, im_ind = fcn_mark_im_ind(ind, 20)
    if mark_ind == 2:
        y.append(0)
    else:
        y.append(1)
    scores = np.mean(nm_model.predict(samps_2), axis=0)
    save_scores.append(scores)

Currently evaluating fold number fulltraining
Train on 5822 samples
Epoch 1/20
5822/5822 [==============================] - 1s 94us/sample - loss: 0.4237 - accuracy: 0.8451
Epoch 2/20
5822/5822 [==============================] - 0s 46us/sample - loss: 0.1662 - accuracy: 0.9589
Epoch 3/20
5822/5822 [==============================] - 0s 47us/sample - loss: 0.1078 - accuracy: 0.9729
Epoch 4/20
5822/5822 [==============================] - 0s 48us/sample - loss: 0.0761 - accuracy: 0.9813
Epoch 5/20
5822/5822 [==============================] - 0s 50us/sample - loss: 0.0565 - accuracy: 0.9856
Epoch 6/20
5822/5822 [==============================] - 0s 49us/sample - loss: 0.0438 - accuracy: 0.9894
Epoch 7/20
5822/5822 [==============================] - 0s 50us/sample - loss: 0.0347 - accuracy: 0.9923
Epoch 8/20
5822/5822 [==============================] - 0s 47us/sample - loss: 0.0289 - accuracy: 0.9935
Epoch 9/20
5822/5822 [==============================] - 0s 50us/sample - loss: 0.0234 - accu

In [143]:
139/150

0.9266666666666666

In [132]:
yhat = []
for s in save_scores:
    if s[0] > 0.29:
        yhat.append(0)
    else:
        yhat.append(1)
acc = np.sum(np.array(yhat)==np.array(y))/len(yhat)
print('Binary accuracy at expected optimal cutoff: {:0.3f}'.format(acc))

Binary accuracy at expected optimal cutoff: 0.950


In [134]:
scpath = os.path.join(basedir, 'TestSetSamples/2019_08_26_ClassifierNetworkForSelection_Scores_0.p')
yhatclass = np.ones(100, dtype=int) * -1
yhatfin = np.ones(100, dtype=int) * 2
yfin = np.ones(100, dtype=int) * -1
with open(scpath, 'rb') as f:
    sc = pickle.load(f)
for ind in range(100):
    mark, im_ind = fcn_mark_im_ind(ind, 20)
    sctemp = np.average(sc[ind], axis=0)
    yhatclass[ind] = np.argmax(sctemp)
    if yhat[ind] != 0:
        yhatfin[ind] = yhatclass[ind] 
    yfin[ind] = mark
print('Overall classification accuracy: {:0.3f}'.format(np.sum(yhatfin==yfin) / len(yfin)))

Overall classification accuracy: 0.900


In [135]:
conf = np.zeros([5, 5])
for ind in range(len(yfin)):
    conf[yfin[ind], yhatfin[ind]] += 1
print(conf)

[[17.  0.  2.  1.  0.]
 [ 0. 17.  0.  1.  2.]
 [ 0.  0. 20.  0.  0.]
 [ 0.  0.  1. 18.  1.]
 [ 0.  0.  2.  0. 18.]]


In [ ]:
smd2ydx2 = oa.finder_network.fcn_smoothed_d2ydx2(samp_array)

In [ ]:
model.predict(np.expand_dims(np.expand_dims(smd2ydx2, axis=0), axis=2))